# Example 23 - Overhung Rotor

This example is based on Example 6.8.1 from {cite}`friswell2010dynamics`.

Consider the model of a simple overhung rotor, 1.5 m long with bearings at 0.0 m and 1.0 m, and shown in Figure 6.49. The bearings are short in that they present insignificant angular stiffness to the shaft, but they present finite translational stiffness. The shaft is 25 mm in diameter and the disk at the overhung end is 250 mm in diameter and 40 mm thick. The shaft and disk are made of steel, and a mass density p = 7,810 kg/m3, a modulus of elasticity E = 211 GPa, and a Poisson’s ratio of 0.3 are assumed. Several different variants of this system are considered in which the differences lie in the bearing (and bearing-support) properties, shown in Table 6.3, and the steady rotational speed of the shaft. Calculate the eigenvalues for various rotor spin speeds and estimate the critical speeds for unbalance excitation.



In [17]:
import ross as rs
import numpy as np
import plotly.graph_objects as go
import pandas as pd

# uncomment the lines below if you are having problems with plots not showing
# import plotly.io as pio
# pio.renderers.default = "notebook"

Q_ = rs.Q_
rs.__version__

'1.4.3'

In [18]:
# Material
steel = rs.Material("steel", E=211e9, Poisson=0.3, rho=7810)

In [19]:
# Cylindrical shaft elements
L =   6 * [250]
i_d = 6 * [0]
o_d = 6 * [25]

shaft_elements = [
    rs.ShaftElement(
        n=i,
        L=Q_(l, "mm"),
        idl=Q_(id1, "mm"),
        odl=Q_(od1, "mm"),
        material=steel,
        shear_effects=True,
        rotary_inertia=True,
        gyroscopic=True,
    )
    for i, l, id1, od1, in zip(range(len(L)), L, i_d, o_d)
]
print("Number os shaft elements: %d." % len(shaft_elements))

Number os shaft elements: 6.


In [20]:
# Creating a disk element
disk_elements = [
    rs.DiskElement(
        n=6,
        m=Q_(15.3, "kg"),
        Id=Q_(0.062, "kg * m**2"),
        Ip=Q_(0.120, "kg * m**2"),
        scale_factor=6,
    )
]

## Characteristic roots computed for a number of configurations

In [21]:
"""
    Table 6.3 - Bearing and Support characteristics
"""

cases = {
    1: [10, 10, 0, 0],
    2: [10, 20, 0, 0],
    3: [10, 20, 60, 60],
    4: [10, 20, 400, 400],
    5: [0.2, 0.4, 0, 0],
}
"""
    Bearing configurations from table 6.4
"""
configurations = [
    [0, 1, 1],
    [0, 2, 2],
    [0, 2, 3],
    [0, 2, 4],
    [0, 5, 5],
    [1e3, 1, 1],
    [1e3, 2, 3],
    [2e3, 2, 3],
    [3e3, 2, 3],
]

# Create empty lists to store data and bearing pairs
data = []
bearing_pairs = []

# Configure Pandas to display only three decimal places
pd.options.display.float_format = "{:.3f}".format

# Use a for loop to generate data
for i in configurations:
    speed, left, rigth = i

    # Bearings instances
    bearing_elements = [
        rs.BearingElement(
            n=0,
            kxx=cases[left][0] * 1e6,
            kyy=cases[left][1] * 1e6,
            cxx=cases[left][2] * 1e3,
            cyy=cases[left][3] * 1e3,
            scale_factor=6,
        ),
        rs.BearingElement(
            n=4,
            kxx=cases[rigth][0] * 1e6,
            kyy=cases[rigth][1] * 1e6,
            cxx=cases[rigth][2] * 1e3,
            cyy=cases[rigth][3] * 1e3,
            scale_factor=6,
        ),
    ]
    rotor = rs.Rotor(shaft_elements, disk_elements, bearing_elements)

    # Saving bearing pairs for later analysis
    bearing_pairs.append(bearing_elements)

    # Performing modal analysis
    modal = rotor.run_modal(speed=Q_(speed, "RPM"))

    # Selection of the first 4 complex roots
    roots = []
    for root in modal.evalues:
        if np.iscomplex(root):
            roots.append(root)
        if len(roots) == 4:
            break

    data.append(
        {
            ("Config:", "Speed"): speed,
            ("Config:", "Left"): left,
            ("Config:", "Rigth"): rigth,
            ("Roots:", "First"): roots[0],
            ("Roots:", "Second"): roots[1],
            ("Roots:", "Third"): roots[2],
            ("Roots:", "Fourth"): roots[3],
        }
    )

multi_index = pd.MultiIndex.from_tuples(
    [
        ("Config:", "Speed"),
        ("Config:", "Left"),
        ("Config:", "Rigth"),
        ("Roots:", "First"),
        ("Roots:", "Second"),
        ("Roots:", "Third"),
        ("Roots:", "Fourth"),
    ]
)

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(data, columns=multi_index)

# Display the DataFrame
df

Config:                    Roots:                                 \
     Speed Left Rigth          First         Second           Third   
0    0.000    1     1 -0.000+44.322j -0.000+44.322j -0.000+397.537j   
1    0.000    2     2 -0.000+44.322j -0.000+44.408j -0.000+397.537j   
2    0.000    2     3 -0.038+44.332j -0.010+44.409j -1.219+400.564j   
3    0.000    2     4 -0.066+44.438j -0.038+44.442j -0.211+401.037j   
4    0.000    5     5 -0.000+37.630j  0.000+40.664j -0.000+226.242j   
5 1000.000    1     1 -0.000+42.250j  0.000+46.404j -0.000+389.801j   
6 1000.000    2     3 -0.020+42.292j -0.028+46.460j -0.617+391.899j   
7 2000.000    2     3 -0.017+40.243j -0.033+48.539j -0.189+377.688j   
8 3000.000    2     3 -0.014+38.241j -0.039+50.592j -0.000+356.968j   

                   
           Fourth  
0  0.000+397.537j  
1 -0.000+400.142j  
2 -0.850+401.182j  
3 -0.209+401.849j  
4  0.000+281.853j  
5  0.000+402.327j  
6 -1.358+406.609j  
7 -1.607+410.454j  
8 -1.788+413.175j

## Rotor Model

In [22]:
rotor = rs.Rotor(shaft_elements, disk_elements, bearing_pairs[4])
print("ROTOR DATA:\nRotor total mass = ", np.round(rotor.m, 2))
print("Rotor center of gravity =", np.round(rotor.CG, 2))
rotor.plot_rotor(width=500, height=400)

ROTOR DATA:
Rotor total mass =  21.05
Rotor center of gravity = 1.3


## Campbell Diagram

The following chart presents the Campbell diagram for a system where both the left and right bearings exhibit the characteristics of Case 5, as detailed in Table 6.3 on page 279 of [Fryswell, 2010](#References).

The chart can be found in page 280 of the reference.

In [23]:
samples = 41
max_spin = 400
speed_range = np.linspace(0, max_spin, samples)
campbell = rotor.run_campbell(speed_range, frequency_type="wn")
fig = campbell.plot(frequency_units="rpm", width=600, height=600)
for i in fig.data:
    try:
        i["y"] = i["y"] / 60
    except:
        pass
fig.update_yaxes(title="Natural Frequencies (Hz)", range=[0, 90])
fig.update_xaxes(title="Rotor Spin Speed (rpm)", range=[0, max_spin * 60 / (2 * np.pi)])

# Modal Shapes

Mode shapes for the critical speeds for the onverhung rotor. This example is based on Example 6.9.1 page 281 from [Fryswell, 2010](#References).

Find the critical speeds and associated mode shapes for the 
system defined in Example 6.8.1 with bearing ccharacteristic case 5, defined in
Table 6.3.

In [24]:
modal = rotor.run_modal(speed=Q_(358.43, "RPM"))
fig = modal.plot_mode_3d(0, frequency_units="rpm", width=600, height=600)
fig.layout.title.text = "FIRST CRITICAL SPEED" + fig.layout.title.text
fig

In [25]:
modal = rotor.run_modal(speed=Q_(389.35, "RPM"))
fig = modal.plot_mode_3d(1, frequency_units="rpm", width=600, height=600)
fig.layout.title.text = "SECOND CRITICAL SPEED" + fig.layout.title.text
fig

In [26]:
modal = rotor.run_modal(speed=Q_(2123.79, "RPM"))
fig = modal.plot_mode_3d(2, frequency_units="rpm", width=600, height=600)
fig.layout.title.text = "THIRD CRITICAL SPEED" + fig.layout.title.text
fig

In [27]:
modal = rotor.run_modal(speed=Q_(2659.10, "RPM"))
fig = modal.plot_mode_3d(3, frequency_units="rpm", width=600, height=600)
fig.layout.title.text = "FOURTH CRITICAL SPEED" + fig.layout.title.text
fig

In [28]:
modal = rotor.run_modal(speed=Q_(3118.14, "RPM"))
fig = modal.plot_mode_3d(4, frequency_units="rpm", width=600, height=600)
fig.layout.title.text = "FIFHT CRITICAL SPEED" + fig.layout.title.text
fig

## References

```{bibliography}
:filter: docname in docnames
```